In [1]:
import json
import matplotlib.pyplot as plt
from collections import Counter
import statistics
from datetime import datetime
from collections import defaultdict
from torchtext.vocab import GloVe
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.tokenize import word_tokenize
import itertools


import torch
from torch import nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter


from sklearn.model_selection import train_test_split



from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

from torchtext.vocab import GloVe



In [2]:
glove = GloVe(name="6B", dim=300)

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_review_text(review_text):
    review_text = review_text.lower()
    review_text = re.sub(r'\W+', ' ', review_text)
    words = [word for word in review_text.split() if word not in stop_words]
    processed_words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    return processed_words


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/outakaratakara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/outakaratakara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
review_train = []
with open("./dataset/review_train.json", "r") as file:
    for line in file:
        review_train.append(json.loads(line))

review_test = []
with open("./dataset/review_test.json", "r") as file:
    for line in file:
        review_test.append(json.loads(line))
        
review_text_train = [
    preprocess_review_text(review['text']) for review in review_train
]

review_text_test = [
    preprocess_review_text(review['text']) for review in review_test
]


In [5]:
all_text = " ".join(itertools.chain(*review_text_train))

In [6]:
rating_train = [review['stars'] for review in review_train]
rating_test = [review['stars'] for review in review_test]

In [ ]:
class NN2DAN(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.embedding = torch.nn.Embedding(len(all_text), 300)
        self.embedding.weight.data.copy_(glove.get_vecs_by_tokens(all_text, lower_case_backup=True))
        self.fc1 = nn.Linear(300, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)
        self.log_softmax = nn.LogSoftmax(dim=1)
            
        
    def forward(self, x):
        x = self.embedding(x)
        mask = (x != 0).float()
        x = torch.sum(x, dim=1) / torch.sum(mask, dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.log_softmax(x)
        return x

: 

In [7]:
def train_epoch(data_loader, model, loss_fn, optimizer):
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(data_loader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_train_loss = train_loss / num_batches
    accuracy = correct / size
    return accuracy, average_train_loss

In [ ]:
model = NN2DAN(hidden_size=100)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
model = NN2DAN(hidden_size=100)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


all_train_accuracy = []
all_test_accuracy = []

for epoch in range(100):
        train_accuracy, train_loss = train_epoch(train_loader, model, loss_fn, optimizer)
        all_train_accuracy.append(train_accuracy)

        test_accuracy, test_loss = eval_epoch(test_loader, model, loss_fn, optimizer)
        all_test_accuracy.append(test_accuracy)

        if epoch % 10 == 9:
            print(f'Epoch #{epoch + 1}: train accuracy {train_accuracy:.3f}, dev accuracy {test_accuracy:.3f}')
    


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)